In [3]:
from __future__ import print_function, division
import numpy as np
import scipy as sp
import pandas as pd

IndentationError: unexpected indent (<ipython-input-2-fa8606eaa88e>, line 2)

In [9]:
#column_names = np.array(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 
#                         'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
#                        'ca', 'thal', 'num'])
data = pd.read_csv('dataset/heart-disease.csv')

In [10]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
